In [36]:
import numpy as np
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import math
import pywt
import scipy
from scipy import ndimage

# Create some random data to be tested for WaveCluster

In [173]:
#Generate random points
#centers = [[1, 1], [-1, -1], [1, -1]]
X, y= make_blobs(n_samples=1000, n_features = 1, cluster_std=0.4, random_state=0)

X = StandardScaler().fit_transform(X)

In [175]:
#plt.scatter(X[:,0],X[:,1])

# Segment the space into cubes and bin the data

In [244]:
#Compute and/or set some variables
levels = 1   #set levels of DWT.  May end up being less
n_bins = 2**8
d = X.shape[1] #number of dimensions

In [245]:
H = np.histogramdd(X, bins=n_bins)
data_quant = H[0]

In [246]:
data_quant

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,
        2.,  3.,  0.,  2.,  5.,  1.,  1.,  3.,  3.,  1.,  3.,  2.,  4.,
        1.,  5.,  3.,  5.,  8.,  5.,  4.,  8.,  7.,  8.,  5.,  7.,  6.,
        8.,  8.,  9.,  6.,  7.,  9.,  5.,  8.,  5., 11.,  9.,  9.,  8.,
        9.,  8., 12.,  5.,  6.,  9.,  7.,  9.,  1.,  6.,  4.,  9.,  8.,
        7.,  9.,  2.,  6.,  8.,  7.,  9.,  5.,  7.,  5.,  3.,  4.,  8.,
       11.,  8.,  7.,  5.,  6.,  6.,  8.,  6.,  4.,  5., 13., 12.,  7.,
        7.,  8., 10.,  9., 12.,  7.,  7., 10.,  6.,  8.,  4.,  9.,  8.,
        9.,  7.,  3.,  5.,  2.,  7.,  4.,  4.,  5.,  5.,  5.,  2.,  3.,
        1.,  1.,  3.,  3.,  1.,  2.,  1.,  1.,  1.,  4.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  5.,  0.,  3.,  1.,  0.,
        1.,  2.,  0.,  3.,  6.,  3.,  5.,  1.,  3.,  6.,  2.,  3

# Compute the DWT

In [247]:
#pywt.wavelist()
#Information : http://wavelets.pybytes.com/

In [248]:
#Select a wavelet.
wave = 'db4'
print(pywt.Wavelet(wave))

Wavelet db4
  Family name:    Daubechies
  Short name:     db
  Filters length: 8
  Orthogonal:     True
  Biorthogonal:   True
  Symmetry:       asymmetric
  DWT:            True
  CWT:            False


In [249]:
max_level = pywt.dwt_max_level(data_len = n_bins , filter_len = pywt.Wavelet(wave).dec_len) #figure out max levels to loop over

In [277]:
#Perform dwt on quantized data.
wp = pywt.wavedecn(data=data_quant, wavelet=wave, level = min(levels,max_level))
#wp = pywt.dwtn(data=data_quant, wavelet=wave)

In [278]:
#I cannot see a way around computing DWT twice in the thresholding step
wp_c = pywt.wavedecn(data=data_quant, wavelet=wave, level = min(levels,max_level))

# Threshold the results of DWT

In [279]:
wp

[array([ 6.37243935e-02, -2.15018581e-01,  1.34572734e+00,  2.30377813e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  2.22856099e-02, -1.23310418e-01,
         4.05264785e-01,  1.64092035e+00, -1.00346238e-01,  2.10815722e+00,
         1.64608593e+00,  4.54965418e+00,  1.96085627e+00,  3.86108903e+00,
         2.65797976e+00,  3.85335181e+00,  4.14039280e+00,  5.60403621e+00,
         8.84281866e+00,  7.68696019e+00,  1.08081653e+01,  8.64833337e+00,
         9.36143956e+00,  1.21197840e+01,  9.88619183e+00,  1.05814491e+01,
         8.47919062e+00,  1.33584635e+01,  1.22768604e+01,  1.24595221e+01,
         1.22256086e+01,  9.42544555e+00,  1.19395940e+01,  5.15981864e+00,
         8.66622029e+00,  1.19563453e+01,  8.10269402e+00,  8.24908248e+00,
         1.12861792e+01,  9.63263347e+00,  5.99944376e+00,  7.11018623e+00,
         1.39198876e+01,  9.11288037e+00,  8.03018843e+00,  1.05605672e+01,
         5.1

In [280]:
#Pick a threshold value.  Compute some details for loops
epsilon = .01
keys = wp[1].keys()  #build keys to loop over

In [281]:
#Threshold the DWT
wp_c[0][abs(wp_c[0])<epsilon] = 0
wp_c[0][abs(wp_c[0])>=epsilon] = 1

for i in range(1,min(levels,max_level)+1):
    for k in keys:
        wp_c[i][k][abs(wp_c[i][k])<epsilon] = 0
        wp_c[i][k][abs(wp_c[i][k])>=epsilon] = 1

In [282]:
wp_c

[array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1.]),
 {'d': array([1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
         1., 1., 1.,

# Find connected components

In [283]:
#Compute the connected components of each thresholed DWT.  Adjacnecy is determined by ``four'' connectivity
component = ndimage.label(wp_c[0])
wp_c[0] = component[0]

for i in range(1,min(levels,max_level)+1):
    for k in keys:
        component = ndimage.label(wp_c[i][k])
        wp_c[i][k] = component[0]

In [284]:
wp_c

[array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 4, 4, 4, 4],
       dtype=int32),
 {'d': array([1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 0, 5, 5, 5, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6,
         6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
         6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 7, 7, 7, 0],
        dtype=int32)}]

# Create Lookup Table

In [290]:
#Can I just mask each piece and run backwards? - ANSWER: NO

# Trash

In [14]:
#Create the bins
#bin_list = []
#for i in list(range(d)):
#    dim_min = math.floor(min(X[:,i]))
#    dim_max = math.ceil(max(X[:,i]))
#    dim_bin = np.linspace(start=dim_min, stop=dim_max, num=n_bins)
#    bin_list.append(dim_bin)
#    
#bin_list = np.stack(bin_list, axis=0)

In [15]:
#Quantize data into the bins
#data_quant = []
#index_key = []
#for b in list(range(d)):
#    inds = np.digitize(X[:,b], bin_list[b])
#    index_key.append(inds)
#    data_quant.append(np.bincount(inds, minlength = n_bins))
#
#index_key = np.stack(index_key,axis=0)
#data_quant = np.stack(data_quant, axis=0)